# Github API Issues Puller
This notebook fetches the Github Issues associated with the input github repository name and issue status. The user must first create a github token for api access and save it to a file called 'config.py' in the same directory as this notebook.
The format for the config file is:
GITHUB_TOKEN = 'XX3f427f3d5f0eb236440dd51cab9d1226a93eXX'



In [2]:
#load modules
import requests
import os
from pprint import pprint
import config #Where to save your Github token
import pandas as pd
import json

In [63]:


#Get user input parameters
repo_name = input("Enter the name of the GH Repo you want Issues collected from:\n example: data-science")
REPO = str(repo_name)
value = input("enter the state of issues you want:\n options: open, closed, all")
params = {"content_type" : "Issue", "state":value}

# Define the input parameters
token = config.GITHUB_TOKEN
acc = "application/vnd.github.inertia-preview+json" #this is the Accept type you must have for working with Projects in the current Github Api
#params = {"content_type" : "Issue", "state":"open"} #notice the content type is for issues and the issue state is open
HEADERS = {'Accept': f'{acc}','Authorization': f'token {token}'}
OWNER = "Thinkful-Ed"
#REPO = "data-science"

ISSUES_DICT = {}
def get_issues_by_repository(repository):
    url = "https://api.github.com/repos/{}/{}/issues".format(OWNER, repository)
    headers = HEADERS
    r = requests.get(
        url=url,
        headers=headers, 
        params=params

    )
    issues = r.json()
    while "next" in r.links:
        print("\tNext Page: {}".format(r.links["next"]["url"]))
        r = requests.get(
            url=r.links["next"]["url"],
            headers=headers,
            params=params
        )
        issues.extend(r.json())
    ISSUES_DICT.update({repository: issues})
    return issues

today = pd.to_datetime('now').date()
issue_dict = get_issues_by_repository(REPO)
df = pd.DataFrame.from_dict(issue_dict)
df['date'] = pd.to_datetime(df['created_at']).dt.date
df['week'] = df['date'].dt.week
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
filename = params['state']+'_issues_'+str(REPO)+'_'+str(today)+'.csv'
df.to_csv(filename, index =False)

	Next Page: https://api.github.com/repositories/152430728/issues?content_type=Issue&state=all&page=2
	Next Page: https://api.github.com/repositories/152430728/issues?content_type=Issue&state=all&page=3
	Next Page: https://api.github.com/repositories/152430728/issues?content_type=Issue&state=all&page=4
	Next Page: https://api.github.com/repositories/152430728/issues?content_type=Issue&state=all&page=5
	Next Page: https://api.github.com/repositories/152430728/issues?content_type=Issue&state=all&page=6
	Next Page: https://api.github.com/repositories/152430728/issues?content_type=Issue&state=all&page=7
	Next Page: https://api.github.com/repositories/152430728/issues?content_type=Issue&state=all&page=8
	Next Page: https://api.github.com/repositories/152430728/issues?content_type=Issue&state=all&page=9
	Next Page: https://api.github.com/repositories/152430728/issues?content_type=Issue&state=all&page=10
	Next Page: https://api.github.com/repositories/152430728/issues?content_type=Issue&state=

## Get Daily Issue counts and save out as csv

In [4]:
#get counts per module
f1 = df.title.str.split("for", expand=True)
#f1.columns = ['type','module'] #sometimes has 2 elements and sometimes has 3
f1.columns = ['type','module','none']
f1.module.value_counts()

 REACT                                                         539
 NODE-POSTGRES                                                 526
 HTML-CSS                                                      306
 wd-html-css                                                   197
 WD-START-CODING                                               131
                                                              ... 
 design                                                          1
 students the glob…                                              1
 3.3.5                                                           1
 Reading JS                                                      1
 Node "Working with the Express response object" Assignment      1
Name: module, Length: 193, dtype: int64

In [9]:
#add module names back to df
df['module'] =f1['module']
#group by date and module
module_date = pd.DataFrame(df[['date','module']].groupby(['date','module']).size()).reset_index()
module_date.columns = ['date','modules','cnt']
module_date

,date,modules,cnt
0,2018-11-07,asset upload,1
1,2018-11-11,`html-css-v1`,1
2,2018-11-12,matting,1
3,2018-11-14,careers branding,1
4,2019-01-04,"""Working with large datasets""",1
...,...,...,...
2422,2020-12-09,zid-pf-javascript-foundations,2
2423,2020-12-10,NODE-POSTGRES,3
2424,2020-12-10,WD-JS-START-CODING,1
2425,2020-12-10,zid-pf-developer-workflow,1


In [10]:
#module_date.to_csv('Daily_issue_counts_by_module_web-dev-programs_2020-12-11.csv', index=False)
module_date.to_csv('Daily_issue_counts_by_module_'+str(REPO)+'_'+str(today)+'.csv', index=False)

## Get Weekly Issues counts

In [3]:
# read in an existing issues csv file from previous code run
df = pd.read_csv('all_issues_web-dev-programs_2020-12-11.csv')
df['date'] = pd.to_datetime(df['date'])
df

,url,repository_url,labels_url,comments_url,events_url,html_url,id,node_id,number,title,...,comments,created_at,updated_at,closed_at,author_association,active_lock_reason,body,performed_via_github_app,pull_request,date
0,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://github.com/Thinkful-Ed/web-dev-program...,761801678,MDU6SXNzdWU3NjE4MDE2Nzg=,5211,User-submitted feedback for NODE-POSTGRES,...,0,2020-12-11T02:21:07Z,2020-12-11T02:21:07Z,NaN,NONE,NaN,\nCatherine Pedersen (cathy.pedersen@gmail.com...,NaN,NaN,2020-12-11
1,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://github.com/Thinkful-Ed/web-dev-program...,761775978,MDExOlB1bGxSZXF1ZXN0NTM2MzkzMTQy,5210,3.5.2 Connecting to Databases with Knex review,...,0,2020-12-11T01:35:54Z,2020-12-11T01:35:54Z,NaN,CONTRIBUTOR,NaN,- Minor typo fix - missing word,NaN,{'url': 'https://api.github.com/repos/Thinkful...,2020-12-11
2,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://github.com/Thinkful-Ed/web-dev-program...,761739936,MDExOlB1bGxSZXF1ZXN0NTM2MzYxOTAx,5209,3.5.1 Knex config review,...,1,2020-12-11T00:24:10Z,2020-12-11T00:28:17Z,NaN,CONTRIBUTOR,NaN,- Minor wording update - wanted to separate t...,NaN,{'url': 'https://api.github.com/repos/Thinkful...,2020-12-11
3,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://github.com/Thinkful-Ed/web-dev-program...,761712032,MDExOlB1bGxSZXF1ZXN0NTM2MzM3ODQ2,5208,Fix links,...,0,2020-12-10T23:31:48Z,2020-12-10T23:33:26Z,2020-12-10T23:33:26Z,COLLABORATOR,NaN,NaN,NaN,{'url': 'https://api.github.com/repos/Thinkful...,2020-12-10
4,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://github.com/Thinkful-Ed/web-dev-program...,761641172,MDExOlB1bGxSZXF1ZXN0NTM2Mjc3NTA0,5207,Applying copyedits,...,0,2020-12-10T21:44:03Z,2020-12-10T21:44:29Z,2020-12-10T21:44:24Z,CONTRIBUTOR,NaN,NaN,NaN,{'url': 'https://api.github.com/repos/Thinkful...,2020-12-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5206,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://github.com/Thinkful-Ed/web-dev-program...,370601209,MDExOlB1bGxSZXF1ZXN0MjIzMjQxMzYw,5,Spec: react component composition,...,0,2018-10-16T13:02:41Z,2018-10-23T10:11:27Z,2018-10-22T13:20:00Z,COLLABORATOR,NaN,NaN,NaN,{'url': 'https://api.github.com/repos/Thinkful...,2018-10-16
5207,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://github.com/Thinkful-Ed/web-dev-program...,370259840,MDExOlB1bGxSZXF1ZXN0MjIyOTc5ODk2,4,Spec: testing components,...,0,2018-10-15T17:13:17Z,2018-11-26T18:56:35Z,2018-11-26T18:56:32Z,CONTRIBUTOR,NaN,Testing components incl

In [6]:
# add week column
df['year'] = df['date'].dt.year
df['week'] = df['date'].dt.week
df['month'] = df['date'].dt.month

In [7]:
df.head()

,url,repository_url,labels_url,comments_url,events_url,html_url,id,node_id,number,title,...,author_association,active_lock_reason,body,performed_via_github_app,pull_request,date,module,year,week,month
0,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://github.com/Thinkful-Ed/web-dev-program...,761801678,MDU6SXNzdWU3NjE4MDE2Nzg=,5211,User-submitted feedback for NODE-POSTGRES,...,NONE,NaN,\nCatherine Pedersen (cathy.pedersen@gmail.com...,NaN,NaN,2020-12-11,NODE-POSTGRES,2020,50,12
1,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://github.com/Thinkful-Ed/web-dev-program...,761775978,MDExOlB1bGxSZXF1ZXN0NTM2MzkzMTQy,5210,3.5.2 Connecting to Databases with Knex review,...,CONTRIBUTOR,NaN,- Minor typo fix - missing word,NaN,{'url': 'https://api.github.com/repos/Thinkful...,2020-12-11,None,2020,50,12
2,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://github.com/Thinkful-Ed/web-dev-program...,761739936,MDExOlB1bGxSZXF1ZXN0NTM2MzYxOTAx,5209,3.5.1 Knex config review,...,CONTRIBUTOR,NaN,- Minor wording update - wanted to separate t...,NaN,{'url': 'https://api.github.com/repos/Thinkful...,2020-12-11,None,2020,50,12
3,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://github.com/Thinkful-Ed/web-dev-program...,761712032,MDExOlB1bGxSZXF1ZXN0NTM2MzM3ODQ2,5208,Fix links,...,COLLABORATOR,NaN,NaN,NaN,{'url': 'https://api.github.com/repos/Thinkful...,2020-12-10,None,2020,50,12
4,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://api.github.com/repos/Thinkful-Ed/web-d...,https://github.com/Thinkful-Ed/web-dev-program...,761641172,MDExOlB1bGxSZXF1ZXN0NTM2Mjc3NTA0,5207,Applying copyedits,...,CONTRIBUTOR,NaN,NaN,NaN,{'url': 'https://api.github.com/repos/Thinkful...,2020-12-10,None,2020,50,12


In [8]:
#group by date and module
module_date = pd.DataFrame(df[['year','week','module']].groupby(['year','week','module']).size()).reset_index()
module_date.columns = ['year','week','modules','cnt']
module_date

,year,week,modules,cnt
0,2018,45,`html-css-v1`,1
1,2018,45,asset upload,1
2,2018,46,careers branding,1
3,2018,46,matting,1
4,2019,1,"""Working with large datasets""",1
...,...,...,...,...
1310,2020,50,main->master,1
1311,2020,50,wd-prep-orientation,1
1312,2020,50,zid-pf-developer-workflow,5
1313,2020,50,zid-pf-javascript-foundations,4


In [160]:
#module_date.to_csv('Weekly_issue_counts_by_module_uxui-programs_2020-12-11.csv')
module_date.to_csv('Weekly_issue_counts_by_module_'+str(REPO)+'_'+str(today)+'.csv', index=False)

In [53]:
#group by date and plot
by_date = pd.DataFrame(df[['date']].groupby(['date']).size()).reset_index()
by_date.columns = ['date','count']
import plotly.express as px

fig = px.line(by_date, x='date', y="count")
fig.show()

In [54]:
#calc change in issue count by date
by_date['dif'] = by_date['count'].diff()

fig = px.bar(by_date, x='date', y="dif")
fig.show()